# Q1


*   In this question we want to use POS-tagged training set to compute for each word the tag that maximizes $p(t|w)$.
*   We will implement a simple tokenizer to deal with sentence boundaries.
*   We start by assuming that all unknown words are NN and compute error rate on known and unknown words.
*   Then write at least five rules to do a better job of tagging unknown words, and show the difference in error rates.


In [ ]:
import re
import math

import nltk
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [ ]:
def generate_dict(_samples):
    """
    Generate a dictionary that captures the count of each (word, tag) combination from a list of samples.

    Args:
    _samples (list): List of tuples containing (word, tag) pairs.

    Returns:
    dict: A dictionary where keys are words and values are lists of dictionaries with 'tag' and 'count' keys.
          Each dictionary in the list represents a unique tag associated with the word and its count.
    """

    '''
    Example:
      _samples = [('apple', 'fruit'), ('banana', 'fruit'), ('apple', 'fruit'), ('apple', 'color'), ('banana', 'color')]
      After calling generate_dict(_samples), the expected output would be:
      {
          'apple': [
              {'tag': 'fruit', 'count': 2},
              {'tag': 'color', 'count': 1}
          ],
          'banana': [
              {'tag': 'fruit', 'count': 1},
              {'tag': 'color', 'count': 1}
          ]
      }
      This represents that 'apple' appeared twice with the tag 'fruit' and once with the tag 'color',
      while 'banana' appeared once with both 'fruit' and 'color' tags.

    '''

    dictionary = {}


    ## Your code here
    # create the dictionary described in the comments
    for word, tag in _samples:
        if word not in dictionary:
            dictionary[word] = [{'tag': tag, 'count': 1}]
        else:
            found = False
            for item in dictionary[word]:
                if item['tag'] == tag:
                    item['count'] += 1
                    found = True
                    break
            if not found:
                dictionary[word].append({'tag': tag, 'count': 1})

    ## End Code
    def get_tag_counts(subitem):
        return subitem['count']

    for item in dictionary:
        # dictionary['banana'] =
        #         {'tag': 'fruit', 'count': 1},
        #         {'tag': 'color', 'count': 1}
        sorted(dictionary[item], key=get_tag_counts, reverse=True)

    return dictionary

In [ ]:
def predict_tag(_test_set, _tag_dict):
    """
    Predicts the tags for a given test set of words based on a provided tag dictionary.

    Args:
    _test_set (list): A list of tuples containing (word, true_tag)=(x, y) pairs to be predicted.
    _tag_dict (dict): A dictionary containing words as keys and lists of dictionaries with 'tag' and 'count' keys as values.

    Returns:
    float: The accuracy of the predictions, calculated as the ratio of correct predictions to the total number of predictions.

    Comments:
    - For unknown words, 'NN' (noun) tag is assigned.
    - Tags are assigned based on the highest count for known words.
      - If there are more than 1 tag for a given word, the tag with the highest count is chosen.
      - If there is only 1 tag available, it is returned directly.
    """

    accuracy = 0
    for item in _test_set: # item = (word, true_tag)
        word = item[0]
        true_tag = item[1]
        if word in _tag_dict:
            prediction = _tag_dict[word][0]['tag']  # Choose the tag with the highest count
        else:
            prediction = 'NN'  # Assume unknown words as noun (NN)
        if prediction == true_tag:
          accuracy = accuracy + 1

    accuracy /= len(_test_set)
    print(f"Assuming that all unknown words are NN")
    print(f">> accuracy: {accuracy}")
    return accuracy


In [ ]:
def predict_tag_with_improvements(_test_set, _tag_dict):
    """
    Predicts the tags for a given test set of words based on a provided tag dictionary, with additional rules for unknown words.

    Args:
    _test_set (list): A list of tuples containing (word, true_tag)=(x, y) pairs to be predicted.
    _tag_dict (dict): A dictionary containing words as keys and lists of dictionaries with 'tag' and 'count' keys as values.

    Returns:
    float: The accuracy of the predictions, calculated as the ratio of correct predictions to the total number of predictions.

    Comments:
    - For unknown words, 'NN' (noun) tag is initially assigned.
    - Additional rules are applied to analyze unknown words and assign more specific tags based on patterns observed in the word:
        - 'VBG' (verb, gerund) for words ending in 'ing'
        - 'NP$' (noun, possessive) for words ending in "'s"
        - 'NNS' (noun, plural) for words ending in 's'
        - 'RB' (adverb) for words ending in 'ly'
        - 'VBN' (verb, past participle) for words ending in 'ed'
        - 'JJ' (adjective) for words matching certain patterns like 'ble', 'ish', 'ful', etc.
        - 'CD' (cardinal numeral) for numeric strings
        - 'NP' (noun, proper singular) for capitalized words
    """

    ## Your code here
    accuracy = 0
    for word, true_tag in _test_set: # _test_set[i] = (word, true_tag) = (x, y)
        if word in _tag_dict:
            prediction = _tag_dict[word][0]['tag'] # Choose the tag with the highest count
        else:
            if word.endswith('ing'):
                prediction = 'VBG'
            elif word.endswith("'s"):
                prediction = 'NP$'
            elif word.endswith('s'):
                prediction = 'NNS'
            elif word.endswith('ly'):
                prediction = 'RB'
            elif word.endswith('ed'):
                prediction = 'VBN'
            elif re.match(r'\w+ble$', word) or re.match(r'\w+ish$', word) or re.match(r'\w+ful$', word):
            # elif re.search(r'(ble|ish|ful)$', word):
                prediction = 'JJ'
            elif word.isdigit():
                prediction = 'CD'
            elif word[0].isupper():
                prediction = 'NP'
            else:
                prediction = 'NN' # # Default to noun (NN) for unknown words

        if prediction == true_tag:
            accuracy += 1

    ## End Code
    accuracy /= len(_test_set)
    print(f"With additional rules for unknown words")
    print(f">> accuracy: {accuracy}")
    return accuracy

In [ ]:
CORPUS = brown.tagged_words(categories='news')
CORPUS_SIZE = len(brown.tagged_words(categories='news'))

CUT_OFF = math.floor(CORPUS_SIZE * 0.75)

# section off training and testing lists from corpus
training_list = CORPUS[:CUT_OFF]
testing_list = CORPUS[CUT_OFF:]

# duplicates are ignored in sets
training_set = set(training_list)
testing_set = set(testing_list)
intersection = training_set.intersection(testing_set)

print(f"length of training set:     {len(training_list)}")
print(f"length of testing set:      {len(testing_list)}")

# uncomment to see how much the training set and testing set overlap
print(f"intersection:               {len(intersection)}")

# uncomment to survey tagged corpus
# print(training_set)

print("-----------------------------------")

tag_dict = generate_dict(training_list)
accurary_base = predict_tag(testing_list, tag_dict)
accurary_impr = predict_tag_with_improvements(testing_list, tag_dict)
delta = math.floor((accurary_impr - accurary_base) * len(testing_list))
print(f"{delta} more words got correctly classified.")

length of training set:     75415
length of testing set:      25139
intersection:               3429
-----------------------------------
Assuming that all unknown words are NN
>> accuracy: 0.8184096423883209
With additional rules for unknown words
>> accuracy: 0.8627630375114365
1115 more words got correctly classified.


In [ ]:
# print(training_set)
list(training_set)[:100]

[('Holbrook', 'NP'),
 ('like', 'VB'),
 ('Norman', 'NP'),
 ('discussing', 'VBG'),
 ('Nautilus', 'NP'),
 ('policemen', 'NNS'),
 ('Explosion', 'NN-HL'),
 ('Johnston', 'NP'),
 ('$47,101,000', 'NNS'),
 ('Golden', 'JJ-TL'),
 ('Price', 'NP'),
 ('Jan.', 'NP'),
 ('allotted', 'VBD'),
 ("Titche's", 'NP$'),
 ('Displayed', 'VBN'),
 ('championship', 'NN'),
 ('vacation', 'NN'),
 ('pinch-hitters', 'NNS'),
 ('Lynn', 'NP'),
 ('desperate', 'JJ'),
 ('spree', 'NN'),
 ('oats', 'NN'),
 ('surgery', 'NN'),
 ('9', 'CD-TL'),
 ('died', 'VBD'),
 ('York-Pennsylvania', 'NP-TL'),
 ('weeks', 'NNS'),
 ('well', 'QL'),
 ('trims', 'NNS'),
 ('Hartford', 'NP'),
 ('chair', 'NN'),
 ('consonance', 'NN'),
 ('Week', 'NN'),
 ('Shrove', 'NP'),
 ('exposed', 'VBD'),
 ('Donnell', 'NP'),
 ('Clayton', 'NP'),
 ('racket', 'NN'),
 ('Snead', 'NP'),
 ('Corcoran', 'NP'),
 ('specialize', 'VB'),
 ('portable', 'JJ'),
 ('Century', 'NN-TL'),
 ('Bontempo', 'NP'),
 ('deeply', 'QL'),
 ('inclined', 'VBN'),
 ('Once', 'RB'),
 ('responding', 'VBG'),
 ('

# comparing the most frequent tag baseline versions.
### 'Assuming that all unknown words are NN' versus 'With additional rules for unknown words' 
- most frequent tag baseline: `accuracy: 0.81841` (assuming all unknown words are 'NN')
- improved most frequent tag baseline: `accuracy: 0.86276` (using the additional rules for unknown words)

# Analysis:

### Steps we have gone through are:
- Data Preparation: The Brown Corpus from the NLTK library is used for training and testing. It's divided into training and testing sets.
- The generate_dict() function creates a dictionary where the keys are unique words, and the values are lists of dictionaries. Each dictionary in the list represents a unique tag associated with the word and its count.
- The predict_tag() function uses the dictionary generated in the previous step to predict the tags for the test set. It assigns the 'NN' (noun) tag to unknown words and selects the tag with the highest count for known words.
- The predict_tag_with_improvements() function builds on the previous one, adding more specific rules to handle unknown words. It checks for certain patterns in the word (e.g., ending in 'ing', 's', 'ly', 'ed', etc.) and assigns more appropriate tags based on these patterns.
- The script first divides the Brown corpus (news category) into a training set (75%) and a testing set (25%). It then generates the tag dictionary from the training set and uses it to predict the tags for the testing set.
- The initial accuracy, assuming all unknown words are 'NN', is printed.
- The improved accuracy, using the additional rules for unknown words, is then printed.
- The delta, which represents the number of more words that were correctly classified with the improved rules, is also printed.

### The key observations from the results are:

- The initial accuracy `which is 0.81841`, assuming all unknown words are 'NN', is a baseline that provides a starting point for comparison.
- The improved accuracy `which is 0.86276`, using the additional rules for unknown words, shows an increase (approximately 5%) in performance.
- The delta value `which is 1115` quantifies the improvement, indicating that the additional rules helped correctly classify a substantial number of words that were previously misclassified.

### This analysis demonstrates the importance of developing more sophisticated rules and techniques to handle unknown words in part-of-speech tagging, which can lead to substantial improvements in the overall accuracy of the system.